In [122]:
import pandas as pd
import os

# Read and merge CSV files into big one

In [217]:
folder_path = "merge_asof"

# Set path to start file whick will be a reference for merging
start_file = "Ilb1.csv"
df0 = pd.read_csv(os.path.join(folder_path, start_file), encoding='utf-8', delimiter=',', header=0)
df_merged = df0.copy()

In [219]:
# Show list of CSV files that will be merged
for filename in os.listdir(folder_path):
    if filename.endswith(".csv") & ~(filename.startswith("test_")): # set conditions to filter files
        print(filename)

Ilb1.csv
Ilb2.csv
Ilb6.csv


C:\Users\Engineer\AppData\Local\Temp\ipykernel_17208\3010354592.py:3: DeprecationWarning: Bitwise inversion '~' on bool is deprecated and will be removed in Python 3.16. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if filename.endswith(".csv") & ~(filename.startswith("test_")): # set conditions to filter files


In [220]:
# Read CSV files and merge them into a single DataFrame
for filename in os.listdir(folder_path):
    if (filename != start_file) & (filename.endswith(".csv")) & ~(filename.startswith("test_")): # set conditions to filter files
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, encoding='utf-8', delimiter=',', header=0)
        df.drop(columns = ['TimeUS'], inplace=True) # Not needed dublicate column
        df_merged = pd.merge_asof(df_merged, df, on='timestamp', tolerance=0.01, direction='nearest')

C:\Users\Engineer\AppData\Local\Temp\ipykernel_17208\218998429.py:3: DeprecationWarning: Bitwise inversion '~' on bool is deprecated and will be removed in Python 3.16. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if (filename != start_file) & (filename.endswith(".csv")) & ~(filename.startswith("test_")): # set conditions to filter files
C:\Users\Engineer\AppData\Local\Temp\ipykernel_17208\218998429.py:3: DeprecationWarning: Bitwise inversion '~' on bool is deprecated and will be removed in Python 3.16. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if (filename != start_file) & (filename.endswith(".c

## Get summary statistics of merged data and save result

In [221]:
print(f"List of columns:\n {df_merged.columns}")

List of columns:
 Index(['timestamp', 'TimeUS', 'Heading', 'Pitch', 'Roll', 'GyrX', 'GyrY',
       'GyrZ', 'AccX', 'AccY', 'AccZ', 'MagX', 'MagY', 'MagZ', 'USW', 'Vinp',
       'Temp', 'Lat', 'Lng', 'Alt', 'ESpd', 'NSpd', 'VSpd', 'AirSpd', 'NWind',
       'EWind', 'NWindStd', 'EWindStd'],
      dtype='object')


In [222]:
df_merged.head(10)

,timestamp,TimeUS,Heading,Pitch,Roll,GyrX,GyrY,GyrZ,AccX,AccY,...,Lng,Alt,ESpd,NSpd,VSpd,AirSpd,NWind,EWind,NWindStd,EWindStd
0,1.749653e+09,73844917,207.889999,6.49,-3.10,0.75748,0.01481,-0.13135,0.113984,0.052751,...,-9.320627,97.708008,0.0,0.0,-0.05,-0.01,0.0,0.0,0.0,0.0
1,1.749653e+09,73854241,207.889999,6.49,-3.09,0.79922,0.01075,-0.13142,0.113707,0.051803,...,-9.320627,97.707008,0.0,0.0,-0.05,-0.01,0.0,0.0,0.0,0.0
2,1.749653e+09,73864746,207.889999,6.49,-3.08,0.70018,-0.04003,-0.06693,0.111788,0.051101,...,-9.320627,97.707008,0.0,0.0,-0.05,-0.01,0.0,0.0,0.0,0.0
3,1.749653e+09,73874859,207.889999,6.49,-3.08,0.79071,-0.02418,-0.06328,0.110342,0.050614,...,-9.320627,97.706001,0.0,0.0,-0.05,-0.01,0.0,0.0,0.0,0.0
4,1.749653e+09,73884128,207.889999,6.49,-3.07,0.73019,0.04813,-0.05778,0.112285,0.050794,...,-9.320627,97.705002,0.0,0.0,-0.05,-0.01,0.0,0.0,0.0,0.0
5,1.749653e+09,73894175,207.889999,6.49,-3.06,0.62104,0.05523,-0.02896,0.112541,0.050806,...,-9.320627,97.705002,0.0,0.0,-0.05,-0.01,0.0,0.0,0.0,0.0
6,1.749653e+09,73904548,207.889999,6.49,-3.06,0.48444,0.08091,-0.00747,0.111923,0.050300,...,-9.320627,97.704002,0.0,0.0,-0.05,-0.01,0.0,0.0,0.0,0.0
7,1.749653e+09,73915165,207.940002,6.49,-3.05,0.34032,0.08610,-0.03545,0.112857,0.050032,...,-9.320627,97.739006,0.0,0.0,-0.04,-0.01,0.0,0.0,0.0,0.0
8,1.749653e+09,73924712,207.940002,6.49,-3.05,0.24344,-0.04227,-0.03029,0.113310,0.048481,...,-9.320627,97.738007,0.0,0.0,-0.05,-0.01,0.0,0.0,0.0,0.0
9,1.749653e+09,73934868,207.940002,6.49,-3.05,0.19767,-0.01540,-0.03970,0.113749,0.048343,...,-9.320627,97.738007,0.0,0.0,-0.05,-0.01,0.0,0.0,0.0,0.0


In [223]:
df_merged.shape

(134667, 28)

In [224]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134667 entries, 0 to 134666
Data columns (total 28 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   timestamp  134667 non-null  float64
 1   TimeUS     134667 non-null  int64  
 2   Heading    134667 non-null  float64
 3   Pitch      134667 non-null  float64
 4   Roll       134667 non-null  float64
 5   GyrX       134667 non-null  float64
 6   GyrY       134667 non-null  float64
 7   GyrZ       134667 non-null  float64
 8   AccX       134667 non-null  float64
 9   AccY       134667 non-null  float64
 10  AccZ       134667 non-null  float64
 11  MagX       134667 non-null  float64
 12  MagY       134667 non-null  float64
 13  MagZ       134667 non-null  float64
 14  USW        134667 non-null  int64  
 15  Vinp       134667 non-null  float64
 16  Temp       134667 non-null  float64
 17  Lat        134667 non-null  float64
 18  Lng        134667 non-null  float64
 19  Alt        134667 non-n

In [225]:
df.describe()

,timestamp,AirSpd,NWind,EWind,NWindStd,EWindStd
count,1.346650e+05,134665.000000,134665.0,134665.0,134665.0,134665.0
mean,1.749653e+09,25.420462,0.0,0.0,0.0,0.0
std,3.888632e+02,14.964227,0.0,0.0,0.0,0.0
min,1.749653e+09,-24.010000,0.0,0.0,0.0,0.0
25%,1.749653e+09,8.929999,0.0,0.0,0.0,0.0
50%,1.749653e+09,32.579998,0.0,0.0,0.0,0.0
75%,1.749654e+09,35.829998,0.0,0.0,0.0,0.0
max,1.749654e+09,50.869999,0.0,0.0,0.0,0.0


## Test how timestamps from different files relative to merged one

In [226]:
# Get random row in merged DataFrame and from particular file to compare

# Define row indices to compare
a, b = 11, 15

# Get random file
random_file = 'Ilb6.csv'  # Replace with a random file name from the folder
df_rf = pd.read_csv(os.path.join(folder_path, random_file), encoding='utf-8', delimiter=',', header=0)

# How data from the random file looks like
df_rf.loc[a:b,:].style.format({'timestamp': '{:.4f}'}) # Format timestamp for display

,timestamp,TimeUS,AirSpd,NWind,EWind,NWindStd,EWindStd
11,1749652624.2278,73955481,-0.010000,0.000000,0.000000,0.000000,0.000000
12,1749652624.2373,73964970,-0.010000,0.000000,0.000000,0.000000,0.000000
13,1749652624.2488,73976500,-0.010000,0.000000,0.000000,0.000000,0.000000
14,1749652624.2575,73985170,-0.010000,0.000000,0.000000,0.000000,0.000000
15,1749652624.2676,73995268,-0.010000,0.000000,0.000000,0.000000,0.000000


In [227]:
# How data from the random file looks like in the merged DataFrame. 'TimeUS' value is shown from the merged DataFrame
df_merged.loc[a:b, df_rf.columns].style.format({'timestamp': '{:.4f}'})

,timestamp,TimeUS,AirSpd,NWind,EWind,NWindStd,EWindStd
11,1749652624.2278,73955432,-0.010000,0.000000,0.000000,0.000000,0.000000
12,1749652624.2373,73964918,-0.010000,0.000000,0.000000,0.000000,0.000000
13,1749652624.2488,73976455,-0.010000,0.000000,0.000000,0.000000,0.000000
14,1749652624.2575,73985119,-0.010000,0.000000,0.000000,0.000000,0.000000
15,1749652624.2676,73995220,-0.010000,0.000000,0.000000,0.000000,0.000000


In [228]:
# Save merged DataFrame to CSV
output_file = "merged_data.csv"
df_merged.to_csv(os.path.join(folder_path, output_file), index=False)